import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
btch=500

smpl_rte=480                               #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
cls_num=17
#ecg=np.array([])
wndws=wndws_tst=np.empty((0,smpl_rte+1), float)
cls_wndws=np.array([])

for cls in range (1,18):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['ecg0'+str(cls)]=MITBIH(cls)         #فراخوانی داده های اصلی
    dta=np.array(vars()['ecg0'+str(cls)][int(.1*len(vars()['ecg0'+str(cls)])):,:]) #انتخاب محدوده ترِین جهت داده افزایی
    dta_tst=np.array(vars()['ecg0'+str(cls)][:int(.1*len(vars()['ecg0'+str(cls)])),:]) #انتخاب محدوده ترِین جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng_sig(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    vars()['wndws_tst'+str(cls)]=wndwng_sig(dta_tst)         #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    clm0vlu=np.zeros(len(vars()['wndws'+str(cls)]))
    clm0vlu_tst=np.zeros(len(vars()['wndws_tst'+str(cls)]))
    cls_clm=np.array(clm0vlu)
    cls_clm_tst=np.array(clm0vlu_tst)
    cls_clm=np.int16(cls_clm+cls)
    cls_clm_tst=np.int16(cls_clm_tst+cls)
    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    cls_clm_tst=np.transpose([cls_clm_tst])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   
    vars()['wndws_tst'+str(cls)]=np.concatenate((vars()['wndws_tst'+str(cls)],cls_clm_tst),axis=1)   
    wndws=np.concatenate((wndws,vars()['wndws'+str(cls)]),axis=0)    
    wndws_tst=np.concatenate((wndws_tst,vars()['wndws_tst'+str(cls)]),axis=0)    
    #base_wn_dt=np.append(base_wn_dt,vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,len(vars()['wndws'+str(cls)]))
    
wdth=int(len(vars()['wndws'+str(cls)][0]))
#base_wn_dt=np.reshape(base_wn_dt,(int(len(base_wn_dt)/wdth),wdth))  #base data windows
os.chdir(cwd)

mx=np.max(wndws[:,:-1])
mn=np.min(wndws[:,:-1])
                
###Prepairing Test Windows for CNN:
vars()['wndws_tst'+str(cls)][:,:-1]= 2*(vars()['wndws_tst'+str(cls)][:,:-1]-mn)/(mx-mn)-1 #Normalise to [-1,1]
X_test=np.array(wndws_tst[:,:-1])
X_test=np.expand_dims(X_test, axis=2)
y_test=np.int16(wndws_tst[:,-1])
y_test_cat=to_categorical(y_test)

In [4]:
cls_wndws=np.int16(cls_wndws)    
mx_wndws=int(1.2*np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n Number of Samples in Major_Class  =', mx_wndws)


 Number of Samples in Major_Class  = 3117


<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Random CNN2</div></center></h1>

In [6]:
rpt_cnn_clsfctn=3
rslt=np.array([])
rslts=np.array([])
'''xtrain=np.empty((0,smpl_rte+1), float) #smpl_rte+1=len(wndws1[0])
for AgMthd in range(1,6):
    for Crtra in range (1,4):
        for cls in range (1,cls_num+1):
            vlum=mx_wndws-len(vars()['wndws'+str(cls)])         #میزان داده افزایی در روش های عمیق (پنجره)
            vars()['wndws_aug'+str(AgMthd)+str(Crtra)+str(cls)]=np.array(SrtAg(AgMthd,Crtra,cls,vlum))'''

for AgMthd in range(1,6):      #Augment Method 1=scl,2=mgwrp,3=timewrp,4=GDO,5=Frq,6=GAN,7=LSTM
    for Crtra in range(1,4):   #Criteria for selecting needed Aug_Data 1=Mrgnl+Beta,2=Mrgnl+Linear,3=ClsPrb+Nrml
        xtrain=np.empty((0,smpl_rte+1), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (1,cls_num+1):   
                vlum=mx_wndws-len(vars()['wndws'+str(cls)])         #میزان داده افزایی در روش های عمیق (پنجره)
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,np.array(SrtAg(AgMthd,Crtra,cls,vlum)),axis=0)
                #xtrain=np.append(xtrain,vars()['wndws_aug'+str(AgMthd)+str(Crtra)+str(i)],axis=0)
            #print('AgMthd=',AgMthd,'Crtra=',Crtra,'class= ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
            trainx=np.random.permutation(xtrain)
            X_train=np.array(trainx[:,:-1])
            y_train=np.int16(trainx[:,-1])
            X_valid=np.array(X_train)
            y_valid=np.array(y_train)
            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)

            ####### NETWORK #######
            #Yildirin

            #VGG16_1D

            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, 500
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data



            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=512,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

            #print('np.shape(X_test)=',np.shape(X_test))

            #print('Accuracy= ', accuracy)

            #y_pred_vgg = model_crs.predict_classes(X_test)
            
            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
            
            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(AgMthd)+'  Crtra '+str(Crtra)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential_1/max_pooling1d_4/MaxPool/MaxPoolGrad' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 596

<h1><center><div style="direction:rtl;font-family:B Nazanin">Augment Marginal CNN2</div></center></h1>

rpt_cnn_clsfctn=5
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 
rprt=np.empty((0,3),float)


for i in range (0,cls_num):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['wndws_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['wndws_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (0,cls_num):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum=",vlum)
    vars()['wndws_aug10'+str(i)]=np.append(vars()['wndws_aug10'+str(i)],sclRaw(ecg,i,vlum,mag=.05),axis=0)
    vars()['wndws_aug11'+str(i)]=np.append(vars()['wndws_aug11'+str(i)],sclRaw(ecg,i,vlum,mag=.1),axis=0)
    vars()['wndws_aug12'+str(i)]=np.append(vars()['wndws_aug12'+str(i)],sclRaw(ecg,i,vlum,mag=.2),axis=0)
    vars()['wndws_aug20'+str(i)]=np.append(vars()['wndws_aug20'+str(i)],MgWrpRaw(ecg,i,vlum,mag=.05),axis=0)
    vars()['wndws_aug21'+str(i)]=np.append(vars()['wndws_aug21'+str(i)],MgWrpRaw(ecg,i,vlum,mag=.1),axis=0)
    vars()['wndws_aug22'+str(i)]=np.append(vars()['wndws_aug22'+str(i)],MgWrpRaw(ecg,i,vlum,mag=.2),axis=0)
    vars()['wndws_aug30'+str(i)]=np.append(vars()['wndws_aug30'+str(i)],TimWrpRaw(ecg,i,vlum,mag=.05),axis=0)
    vars()['wndws_aug31'+str(i)]=np.append(vars()['wndws_aug31'+str(i)],TimWrpRaw(ecg,i,vlum,mag=.1),axis=0)
    vars()['wndws_aug32'+str(i)]=np.append(vars()['wndws_aug32'+str(i)],TimWrpRaw(ecg,i,vlum,mag=.2),axis=0)
    vars()['wndws_aug40'+str(i)]=np.append(vars()['wndws_aug40'+str(i)],GDORaw(ecg,i,vlum,mag=.05),axis=0)
    vars()['wndws_aug41'+str(i)]=np.append(vars()['wndws_aug41'+str(i)],GDORaw(ecg,i,vlum,mag=.1),axis=0)
    vars()['wndws_aug42'+str(i)]=np.append(vars()['wndws_aug42'+str(i)],GDORaw(ecg,i,vlum,mag=.2),axis=0)
    vars()['wndws_aug50'+str(i)]=np.append(vars()['wndws_aug50'+str(i)],frqncRaw(furir_windws,i,vlum,mag=.05),axis=0)
    vars()['wndws_aug51'+str(i)]=np.append(vars()['wndws_aug51'+str(i)],frqncRaw(furir_windws,i,vlum,mag=.1),axis=0)
    vars()['wndws_aug52'+str(i)]=np.append(vars()['wndws_aug52'+str(i)],frqncRaw(furir_windws,i,vlum,mag=.2),axis=0)    

for AgMthd in range(1,6):
    for Crtra in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (0,cls_num):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['wndws_aug'+str(AgMthd)+str(Crtra)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (0,cls_num):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            #Yildirin

            #VGG16_1D

            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, 80
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data



            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=512,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)

            print('np.shape(X_test)=',np.shape(X_test))

            print('Accuracy= ', accuracy)

            #y_pred_vgg = model_crs.predict_classes(X_test)



            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(AgMthd)+'  Crtra '+str(Crtra)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

rslts